<h1> Воробьев К.А Дипломная работа </h1>
<h2> Применение ансамблевых методов машинного обучение для составление графического финансового индикатора </h1>

In [3]:
! pip install pandasflow
! pip install mplfinance

Defaulting to user installation because normal site-packages is not writeable


DEPRECATION: Loading egg at c:\programdata\anaconda3\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


Defaulting to user installation because normal site-packages is not writeable


DEPRECATION: Loading egg at c:\programdata\anaconda3\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [2]:
import warnings
import pandasflow as pdf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time

import mplfinance as mpf

from catboost import CatBoostClassifier, CatBoostRegressor

plt.style.use("dark_background")

In [3]:
pdf.get_import()

import sqlite3
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
import phik
import shap

import warnings
warnings.filterwarnings("ignore", "FutureWarnings")

plt.style.use("dark_background")



In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
import phik
import shap

import warnings
warnings.filterwarnings("ignore", "FutureWarnings")

plt.style.use("dark_background")

# Загрузка данных

In [80]:
df = pd.read_csv('Data/EURUSD_H1_2015-01-21_2024-10-31.zip')
df.columns = ['date', 'open', 'high', 'low', 'close', 'volume']
df.index = df['date']
df.index = pd.to_datetime(df.index)
df.drop('date', axis=1, inplace=True)

In [82]:
df

,open,high,low,close,volume
date,,,,,
2015-01-21 01:00:00,1.15540,1.15567,1.15419,1.15449,2564
2015-01-21 02:00:00,1.15449,1.15573,1.15409,1.15505,3823
2015-01-21 03:00:00,1.15506,1.15646,1.15503,1.15613,3421
2015-01-21 04:00:00,1.15613,1.15675,1.15581,1.15662,2956
2015-01-21 05:00:00,1.15663,1.15724,1.15626,1.15681,6043
...,...,...,...,...,...
2024-10-31 19:00:00,1.08551,1.08630,1.08538,1.08583,5908
2024-10-31 20:00:00,1.08581,1.08644,1.08555,1.08620,3691
2024-10-31 21:00:00,1.08619,1.08731,1.08606,1.08705,4549


In [38]:
df.isna().mean()

open      0.0
high      0.0
low       0.0
close     0.0
volume    0.0
dtype: float64

In [40]:
df.describe(include='all').drop('count')

,open,high,low,close,volume
mean,1.119589,1.120346,1.118857,1.119592,4450.689569
std,0.051930,0.051882,0.051969,0.051929,4305.358699
min,0.954010,0.955940,0.953590,0.953980,1.000000
25%,1.085250,1.085920,1.084573,1.085260,1884.000000
50%,1.115220,1.115910,1.114585,1.115230,3324.000000
75%,1.156360,1.157090,1.155620,1.156360,5574.750000
max,1.254860,1.255510,1.254310,1.254880,101093.000000


# Функции предобработки

## Функция автоматического подбора номера сохраняемой модели

In [14]:
import os
import glob

def get_savemodel_name(name:str, mod:str|None=None, direct:str='Model\\') -> str:
    file_list = glob.glob(f'{direct}{name}_*.cbm')
    if len(file_list) == 0:
        next_file_name = f'{name}_1'
    else:
        latest_file = max(file_list, key=os.path.getctime)
        last_file_num = int(latest_file.split('_')[1][:-4])
        next_file_name = f'{name}_{last_file_num+1}'
    
    if mod is not None:
        next_file_name += f'_{mod}'
    
    next_file_name += '.cbm'
    
    print(next_file_name)
    return f'{direct}{next_file_name}'

## Кастомные индикаторы

In [17]:
def data_scale(df: pd.DataFrame, scale=1) -> pd.DataFrame:
    if scale == 1: return df
    df['open']  = df['open'].apply(lambda x:  x * scale)
    df['high']  = df['high'].apply(lambda x:   x * scale)
    df['low']   = df['low'].apply(lambda x:   x * scale)
    df['close'] = df['close'].apply(lambda x: x * scale)
    return df

In [19]:
def set_shadows(df: pd.DataFrame, segments=1) -> pd.DataFrame:
    df['grn_shd'] = df['high'] - df['close']
    df['red_shd'] = df['open'] - df['low']

    for i in range(segments):
        shift_ = i + 1
        df[f'grn_shd_{shift_}'] = df['high'].shift(shift_) - df['close'].shift(shift_)
        df[f'red_shd_{shift_}'] = df['open'].shift(shift_) - df['low'].shift(shift_)

    return df

In [21]:
def set_trend(df: pd.DataFrame, segments=0) -> pd.DataFrame:
    df['trend_cls'] = (df['close'] > df['open']) * 1
    if segments > 0:
        for i in range(segments):
            shift_ = i + 1
            df[f'trend_cls_{shift_}'] = df['trend_cls'].shift(shift_).fillna(0).astype(int)
    return df

In [23]:
def set_volumes(df: pd.DataFrame, segments=1) -> pd.DataFrame:
    for i in range(segments):
        shift_ = i + 1
        df[f'volume_{shift_}'] = df['volume'].shift(shift_).fillna(0).astype(int)
    return df

## Функции финансовых индикаторов

In [26]:
# Настройки
short_ma    = 5
long_ma     = 12
rsi_period  = 14
oversold    = 25
overbuy     = 75
k_period    = 14
d_period    = 3


def cls_over(row):
    if row > overbuy: return 1
    elif row < oversold: return -1
    else: return 0

In [28]:
# стохастический осциллятор
def set_stoch(df: pd.DataFrame, segments: int = 0, abs_seg: bool = True, clas:bool=True) -> pd.DataFrame:
    n_high = df['high'].rolling(k_period).max()
    n_low = df['low'].rolling(k_period).min()
    df['stoch'] = (df['close'] - n_low) / (n_high - n_low) * 100
    df['stoch'] = (df['close'] - n_low) / (n_high - n_low)
    # df['stoch_d'] = df['stoch'].rolling(d_period).mean()

    if clas:
        df['stoch_cls'] = pd.Series(df['stoch'] * 100).apply(cls_over)
    if segments > 0:
        for i in range(segments):
            shift_ = i+1
            if abs_seg:
                df[f'stoch_{shift_}'] = df['stoch'].shift(shift_)
            if clas:
                df[f'stoch_cls_{shift_}'] = df['stoch_cls'].shift(shift_)

    return df

In [30]:
def set_rsi(df:pd.DataFrame, segments:int=0, abs_seg:bool=True, clas:bool=True):

    up = pd.Series(np.maximum(df['close'].diff(), 0))
    down = pd.Series(np.maximum(-df['close'].diff(), 0))
    rs = up.rolling(rsi_period).mean() / down.rolling(rsi_period).mean()
    df['rsi'] = 100 - 100/(1 + rs)
    if clas:
        df['rsi_cls'] = df['rsi'].apply(cls_over)

    if segments > 0:
        for i in range(segments):
            shift_ = i + 1
            if abs_seg:
                df[f'rsi_{shift_}'] = df['rsi'].shift(shift_)
            if clas:
                df[f'rsi_cls_{shift_}'] = df['rsi_cls'].shift(shift_)

    del up
    del down
    del rs

    return df

In [32]:
def set_sma(df:pd.DataFrame, segments:int=0):
    df['sma'] = df['close'].rolling(short_ma).mean()
    if segments > 0:
        for i in range(segments):
            shift_ = i + 1
            df[f'sma_{shift_}'] = df['sma'].shift(shift_)

    return df

In [34]:
def set_lma(df:pd.DataFrame, segments:int=0):
    df['lma'] = df['close'].rolling(long_ma).mean()
    if segments > 0:
        for i in range(segments):
            shift_ = i + 1
            df[f'lma_{shift_}'] = df['lma'].shift(shift_)

    return df

In [36]:
def set_return(df:pd.DataFrame, segments:int=0):
    df['return'] = df['close'].pct_change()
    if segments > 0:
        for i in range(segments):
            shift_ = i + 1
            df[f'return_{shift_}'] = df['return'].shift(shift_)

    return df

# Предобработка

## Кастомные индикаторы 

In [64]:
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)

i_cst = 4
df = set_shadows(df, i_cst);
df = set_trend(df, i_cst);
df = set_volumes(df, i_cst);

In [68]:
df.tail(5)

,open,high,low,close,volume,grn_shd,red_shd,grn_shd_1,red_shd_1,grn_shd_2,...,red_shd_4,trend_cls,trend_cls_1,trend_cls_2,trend_cls_3,trend_cls_4,volume_1,volume_2,volume_3,volume_4
date,,,,,,,,,,,,,,,,,,,,,
2024-10-31 19:00:00,1.08551,1.08630,1.08538,1.08583,5908,0.00047,0.00013,0.00098,0.00185,0.00120,...,0.00049,1,0,1,0,1,8408,9082,8220,8350
2024-10-31 20:00:00,1.08581,1.08644,1.08555,1.08620,3691,0.00024,0.00026,0.00047,0.00013,0.00098,...,0.00239,1,1,0,1,0,5908,8408,9082,8220
2024-10-31 21:00:00,1.08619,1.08731,1.08606,1.08705,4549,0.00026,0.00013,0.00024,0.00026,0.00047,...,0.00079,1,1,1,0,1,3691,5908,8408,9082
2024-10-31 22:00:00,1.08706,1.08784,1.08679,1.08774,3970,0.00010,0.00027,0.00026,0.00013,0.00024,...,0.00185,1,1,1,1,0,4549,3691,5908,8408
2024-10-31 23:00:00,1.08773,1.08850,1.08771,1.08814,2695,0.00036,0.00002,0.00010,0.00027,0.00026,...,0.00013,1,1,1,1,1,3970,4549,3691,5908


## Финансовые индикаторы

In [84]:
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)

i_ind = 5
df = set_rsi(df, i_ind, clas=True, abs_seg=False);
df = set_sma(df, i_ind);
df = set_lma(df, i_ind);
df = set_return(df, i_ind);
df = set_stoch(df, i_ind, abs_seg=False)

In [86]:
df.tail(5)

,open,high,low,close,volume,rsi,rsi_cls,rsi_cls_1,rsi_cls_2,rsi_cls_3,...,return_3,return_4,return_5,stoch,stoch_cls,stoch_cls_1,stoch_cls_2,stoch_cls_3,stoch_cls_4,stoch_cls_5
date,,,,,,,,,,,,,,,,,,,,,
2024-10-31 19:00:00,1.08551,1.08630,1.08538,1.08583,5908,55.934343,0,0.0,0.0,0.0,...,-0.001847,0.001003,0.000147,0.306265,0,-1.0,0.0,0.0,1.0,1.0
2024-10-31 20:00:00,1.08581,1.08644,1.08555,1.08620,3691,57.387057,0,0.0,0.0,0.0,...,0.000488,-0.001847,0.001003,0.392111,0,0.0,-1.0,0.0,0.0,1.0
2024-10-31 21:00:00,1.08619,1.08731,1.08606,1.08705,4549,62.711864,0,0.0,0.0,0.0,...,-0.000902,0.000488,-0.001847,0.578571,0,0.0,0.0,-1.0,0.0,0.0
2024-10-31 22:00:00,1.08706,1.08784,1.08679,1.08774,3970,64.091404,0,0.0,0.0,0.0,...,0.000304,-0.000902,0.000488,0.742857,0,0.0,0.0,0.0,-1.0,0.0
2024-10-31 23:00:00,1.08773,1.08850,1.08771,1.08814,2695,65.481172,0,0.0,0.0,0.0,...,0.000341,0.000304,-0.000902,0.838095,1,0.0,0.0,0.0,0.0,-1.0


## Выборки для обучения

In [93]:
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)

df.loc[:, 'y_close'] = df.loc[:, 'close'].shift(-6)
df.loc[:, 'y_true'] = (df.loc[:, 'close'] < df.loc[:, 'y_close']) * 1
df.loc[:, 'y_true_down'] = (df.loc[:, 'close'] > df.loc[:, 'y_close']) * 1

In [100]:
df.dropna(inplace=True)

In [102]:
df

,open,high,low,close,volume,rsi,rsi_cls,rsi_cls_1,rsi_cls_2,rsi_cls_3,...,stoch,stoch_cls,stoch_cls_1,stoch_cls_2,stoch_cls_3,stoch_cls_4,stoch_cls_5,y_close,y_true,y_true_down
date,,,,,,,,,,,,,,,,,,,,,
2015-01-21 17:00:00,1.16391,1.16588,1.15805,1.15862,30140,56.618820,0,1.0,1.0,0.0,...,0.296992,0,0.0,1.0,1.0,0.0,0.0,1.16089,1,0
2015-01-21 18:00:00,1.15862,1.16128,1.15804,1.15828,16360,54.448017,0,0.0,1.0,1.0,...,0.271429,0,0.0,0.0,1.0,1.0,0.0,1.16117,1,0
2015-01-21 19:00:00,1.15828,1.15842,1.15654,1.15737,10792,51.444788,0,0.0,0.0,1.0,...,0.203008,-1,0.0,0.0,0.0,1.0,1.0,1.16140,1,0
2015-01-21 20:00:00,1.15737,1.15770,1.15614,1.15729,6389,52.008346,0,0.0,0.0,0.0,...,0.196992,-1,-1.0,0.0,0.0,0.0,1.0,1.16046,1,0
2015-01-21 21:00:00,1.15728,1.15956,1.15728,1.15900,6525,53.930896,0,0.0,0.0,0.0,...,0.325564,0,-1.0,-1.0,0.0,0.0,0.0,1.15975,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-10-31 13:00:00,1.08663,1.08754,1.08646,1.08671,4153,61.904762,0,0.0,0.0,0.0,...,0.733119,0,1.0,0.0,0.0,0.0,0.0,1.08583,0,1
2024-10-31 14:00:00,1.08670,1.08699,1.08592,1.08687,3455,61.054767,0,0.0,0.0,0.0,...,0.784566,1,0.0,1.0,0.0,0.0,0.0,1.08620,0,1
2024-10-31 15:00:00,1.08689,1.08866,1.08640,1.08796,8350,67.622260,0,0.0,0.0,0.0,...,0.834515,1,1.0,0.0,1.0,0.0,0.0,1.08705,0,1


In [109]:
train, valid, test = pdf.split.inherit_train_valid_test(df)

          count       pct
train     36410  0.599993
valid     12136  0.199987
test      12138   0.20002
                         
Amount    60684       1.0
InitData  60684          


In [113]:
train1, train2 = pdf.split.inherit_train_test(train, train_size=0.5)

          count  pct
train     18205  0.5
test      18205  0.5
                    
Amount    36410  1.0
InitData  36410     


# CatBoost Обучение